In [ ]:
! mkdir /content/data
! ls /

bin			  datalab  lib64       root		  tmp
boot			  dev	   media       run		  tools
content			  etc	   mnt	       sbin		  usr
data			  home	   opt	       srv		  var
data.data-00000-of-00001  lib	   proc        sys
data.index		  lib32    python-apt  tensorflow-1.15.2


In [ ]:
COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  local_path = '/content/drive/My Drive/github/DeepLDSIR'
else:
  local_path = r'../'

import os
os.chdir(local_path)
os.environ["CUDA_VISIBLE_DEVICES"]= '0'

import sys
sys.path.insert(0, './models')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
from dataset import get_pipeline, DataGen

data_path = "/content/drive/My Drive/hdsp/data/Arad_dataset/train"
val_path = "/content/drive/My Drive/hdsp/data/Arad_dataset/test"

dataset_params = dict(
    input_size = (512, 512, 31),
    batch_size = 9,
    buffer_size = 1,
    cache_dir = '/content/data',
)

val_params = dict(
    input_size = (512, 512, 31),
    batch_size = 5,
    buffer_size = 1,
    cache_dir = '',
)


train_ds = get_pipeline(data_path, **dataset_params)
val_ds = get_pipeline(val_path, **val_params)

In [ ]:
"""
AUTOTUNE = tf.data.AUTOTUNE
input_size = (512, 512, 31)

dataset = DataGen(input_size=input_size, data_path = data_path)

map_fun = lambda x:  (x, x)

cache_dir = ''
batch_size = 9
buffer_size = 2 

dataset = (
    dataset
    .cache(cache_dir)
    .shuffle(batch_size) # cache_dir='' guarda el cache en RAM
    .batch(batch_size, drop_remainder=True)
    .map(map_fun, num_parallel_calls=AUTOTUNE)
    .prefetch(buffer_size)
)
"""

"\nAUTOTUNE = tf.data.AUTOTUNE\ninput_size = (512, 512, 31)\n\ndataset = DataGen(input_size=input_size, data_path = data_path)\n\nmap_fun = lambda x:  (x, x)\n\ncache_dir = ''\nbatch_size = 9\nbuffer_size = 2 \n\ndataset = (\n    dataset\n    .cache(cache_dir)\n    .shuffle(batch_size) # cache_dir='' guarda el cache en RAM\n    .batch(batch_size, drop_remainder=True)\n    .map(map_fun, num_parallel_calls=AUTOTUNE)\n    .prefetch(buffer_size)\n)\n"

In [ ]:
import os
import tensorflow as tf
import numpy as np
import scipy.io as sio

from utils import *
from models.Alpha8L import *
from metrics import prior_loss, psnr
# from admm import ADMM as algADMM

input_size = (512, 512, 31)
coded_aperture = np.load('ca.npy')

main_model = HSI_net(coded_aperture, input_size=input_size,  pretrained_weights=None)
autoencoder = main_model.priorA

model_params = {
    'loss' : ['mse' , prior_loss],
    'loss_weights' : [1, 0.1],
    'optimizer' : tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=False),
    'metrics' : {'decode' : psnr},
}


autoencoder.compile(**model_params)
autoencoder.summary()

Model: "PriorAutoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512, 512, 31)]    0         
                                                                 
 conv1 (Conv2D)              (None, 512, 512, 64)      17920     
                                                                 
 conv2 (Conv2D)              (None, 512, 512, 64)      36928     
                                                                 
 conv3 (Conv2D)              (None, 512, 512, 32)      18464     
                                                                 
 conv4 (Conv2D)              (None, 512, 512, 32)      9248      
                                                                 
 conv5 (Conv2D)              (None, 512, 512, 16)      4624      
                                                                 
 encode (Conv2D)             (None, 512, 512, 8)  

In [ ]:
! ls

admm.py  data	     demo_recons.ipynb	ldsr	__pycache__  weights
ca.npy	 dataset.py  demo_train.ipynb	models	README.md


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

weights_path = os.path.join('.', 'weights','Alpha8L_Arad.h5')

callbacks = [
    ReduceLROnPlateau(monitor="val_decode_psnr", factor=0.9, patience=15,
                      verbose=1, mode="max",) ,
    ModelCheckpoint(weights_path, monitor='val_decode_psnr', verbose=1,
                    save_best_only=True, save_weights_only=True, mode='max')
]

autoencoder.load_weights(weights_path)
autoencoder.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=callbacks)

Epoch 1/100
     50/Unknown - 572s 11s/step - loss: 0.0037 - decode_loss: 0.0037 - TV_loss: 2.8211e-04 - decode_psnr: 28.4631
Epoch 00001: val_decode_psnr improved from -inf to 31.42926, saving model to ./weights/Alpha8L_Arad.h5
50/50 [==============================] - 591s 11s/step - loss: 0.0037 - decode_loss: 0.0037 - TV_loss: 2.8211e-04 - decode_psnr: 28.4631 - val_loss: 0.0013 - val_decode_loss: 0.0012 - val_TV_loss: 4.5667e-04 - val_decode_psnr: 31.4293 - lr: 0.0010
Epoch 2/100
50/50 [==============================] - ETA: 0s - loss: 8.2923e-04 - decode_loss: 8.0577e-04 - TV_loss: 1.8858e-04 - decode_psnr: 33.3751
Epoch 00002: val_decode_psnr improved from 31.42926 to 32.70102, saving model to ./weights/Alpha8L_Arad.h5
50/50 [==============================] - 293s 6s/step - loss: 8.2923e-04 - decode_loss: 8.0577e-04 - TV_loss: 1.8858e-04 - decode_psnr: 33.3751 - val_loss: 9.5196e-04 - val_decode_loss: 9.1005e-04 - val_TV_loss: 3.7304e-04 - val_decode_psnr: 32.7010 - lr: 0.0010
Ep